In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np

import xgboost as xgb
import pickle

In [2]:
r16_tab1 = pd.read_excel('C:\\Users\\user\\deep_learning\\MDMS\\DataCompters\\R16\\R16_Tfo1_2022-11-15_19\\R16_Tfo1_2022-11-15.xlsx')

In [3]:
r16_tab1

,type_mesure,phase,mesure,date_heure_mesure
0,tension_n,2,236.29,2022-11-14 23:59:45
1,tension_n,1,236.53,2022-11-14 23:59:45
2,courant,2,92.18,2022-11-14 23:59:45
3,desequilibre_tension,1,0.03,2022-11-14 23:59:45
4,tension_ph_hta,2,13630.30,2022-11-14 23:59:45
...,...,...,...,...
391472,tension_n,3,231.52,2022-11-15 23:58:40
391473,tension_ph_hta,3,13354.02,2022-11-15 23:58:40
391474,tension_ph,3,401.80,2022-11-15 23:58:40
391475,desequilibre_courant,3,8.23,2022-11-15 23:58:40


In [12]:
set(r16_tab1.type_mesure)

{'courant',
 'courant_hta',
 'desequilibre_courant',
 'desequilibre_tension',
 'energie',
 'facteur_puissance',
 'frequence',
 'max_courant',
 'max_puissance_active',
 'max_puissance_apparente',
 'max_puissance_reactive',
 'max_tension_n',
 'max_tension_ph',
 'min_courant',
 'min_puissance_apparente',
 'min_puissance_reactive',
 'min_tension_n',
 'min_tension_ph',
 'puissance_active',
 'puissance_apparente',
 'puissance_reactive',
 'taux_de_charge',
 'tdh_courant',
 'tdh_tension',
 'temperature_point_chaud_bt',
 'temperature_tranfo',
 'temps_indisponibilite',
 'tension_n',
 'tension_ph',
 'tension_ph_hta'}

In [19]:
r16_tab1.loc[r16_tab1['type_mesure']=='min_puissance_apparente']



,type_mesure,phase,mesure,date_heure_mesure
57242,min_puissance_apparente,NaN,0.0,2022-11-15 03:29:25
154389,min_puissance_apparente,NaN,0.0,2022-11-15 09:29:25
251185,min_puissance_apparente,NaN,0.0,2022-11-15 15:29:25
251286,min_puissance_apparente,NaN,0.0,2022-11-15 15:29:29
349949,min_puissance_apparente,NaN,0.0,2022-11-15 21:29:25


In [20]:
r16_tab1=r16_tab1.loc[r16_tab1.type_mesure.isin(['max_courant', 'min_tension_n', 'max_tension_n', 
                                                 'max_tension_ph', 'min_puissance_reactive', 'max_puissance_reactive',
                                                 'max_puissance_active', 'max_puissance_apparente', 'min_courant', 'min_tension_ph',
                                                 'min_puissance_active'
                                                 ])]
                                               

In [21]:
mon_dictionnaire = {'max_courant': int(1), 'max_tension_n': int(2), 'max_tension_ph':int(3),  
                    'max_puissance_reactive':int(4), 'max_puissance_active':int(5), 'max_puissance_apparente':int(6), 
                    'min_courant':int(7), 'min_tension_n':int(8), 'min_tension_ph':int(9), 'min_puissance_reactive':int(10),
                    'min_puissance_apparente':int(11)
                    }


r16_tab1 = r16_tab1.set_index('date_heure_mesure')
r16_tab1.index = pd.to_datetime(r16_tab1.index)


def create_features(r16_tab1):
            """
            Create time series features based on time series index.
            """
            r16_tab1 = r16_tab1.copy()
            r16_tab1['heure_mesure'] = r16_tab1.index.hour
        #     r16_tab1['minute_mesure'] = (r16_tab1.index.minute)
        #     r16_tab1['seconde_mesure'] = r16_tab1.index.second
            # r16_tab1['depart'] =r16_tab1.depart
            r16_tab1['type_mesure'] = r16_tab1.type_mesure
            r16_tab1['phase'] = r16_tab1.phase
            return r16_tab1

for type_mesure, encoder in mon_dictionnaire.items():
        # print("l'élément de clé", type_mesure, "vaut", encoder)
        r16_tab1_2 = r16_tab1.copy()

        r16_tab1_2['type_mesure'] = r16_tab1_2['type_mesure'].map({
                             type_mesure : encoder,
                            
                             },
                             na_action=None)

        r16_tab1_2['phase']= r16_tab1_2['phase'].replace(['N', np.nan],[4, 5])
        # set(r16_tab1.phase)
        r16_tab1_2 = r16_tab1_2.dropna(subset='type_mesure')
        r16_tab1_2['phase']=r16_tab1_2['phase'].astype(int)


        train = r16_tab1_2
        test = r16_tab1_2

        r16_tab1_2 = create_features(r16_tab1_2)
        train = create_features(train)
        test = create_features(test)

        FEATURES = ['type_mesure', 'phase', 'heure_mesure']
        TARGET = 'mesure'

        X_train = train[FEATURES]
        y_train = train[TARGET]

        X_test = test[FEATURES]
        y_test = test[TARGET]

        reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                            n_estimators=10000,
                            early_stopping_rounds=50,
                            objective='reg:linear',
                            max_depth=3,
                            learning_rate=0.001)
        reg.fit(X_train, y_train,
                eval_set=[(X_train, y_train), (X_test, y_test)],
                verbose=100)
                
        filename = f'r16_Tfo1_{type_mesure}'

        with open(f'{filename}.pkl', "wb") as f:
            pickle.dump(reg, f)

[12:27:26] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.45256	validation_1-rmse:0.45256
[100]	validation_0-rmse:0.41236	validation_1-rmse:0.41236
[200]	validation_0-rmse:0.37592	validation_1-rmse:0.37592
[300]	validation_0-rmse:0.34290	validation_1-rmse:0.34290
[400]	validation_0-rmse:0.31301	validation_1-rmse:0.31301
[500]	validation_0-rmse:0.28598	validation_1-rmse:0.28598
[600]	validation_0-rmse:0.26154	validation_1-rmse:0.26154
[700]	validation_0-rmse:0.23948	validation_1-rmse:0.23948
[800]	validation_0-rmse:0.21958	validation_1-rmse:0.21958
[900]	validation_0-rmse:0.20168	validation_1-rmse:0.20168
[1000]	validation_0-rmse:0.18558	validation_1-rmse:0.18558
[1100]	validation_0-rmse:0.17070	validation_1-rmse:0.17070
[1200]	validation_0-rmse:0.15700	validation_1-rmse:0.15700
[1300]	validat

In [ ]:
mon_dictionnaire = {'courant': int(1), 'courant_hta': int(2), 'desequilibre_courant':int(3),  
                    'tension_n':int(4), 'tension_ph':int(5), 'tension_ph_hta':int(6), 
                    'desequilibre_tension':int(7)}


In [ ]:
r16_tab1

In [ ]:
test = r16_tab1.copy()

test['type_mesure'] = test['type_mesure'].map({
                             'courant' : int(1),                           
                             },
                             na_action=None)

test['phase']= test['phase'].replace(['N'],[4])
        # set(r16_tab1.phase)
test = test.dropna(subset='type_mesure')
test['phase']=test['phase'].astype(int)
test = create_features(test)
X_test = test[FEATURES]

with open('r16_Tfo1_courant.pkl', "rb") as f:
    reg = pickle.load(f)

test['prediction'] = reg.predict(X_test)
test.iloc[0:60]

In [ ]:
voir_test = test.loc[(test['type_mesure'] == 1) & (test['phase'] == 1)]

a = voir_test.index
b = voir_test.prediction
# c = voir_test.index
# d = voir_test.prediction

# plt.figure(figsize=(12, 8))
# plt.subplot(3,1,1)
plt.plot(a, b, c='yellow')
# plt.plot(c,d, c='yellow')
plt.show

In [ ]:
# r16_tab1 = r16_tab1.set_index('date_heure_mesure')
# r16_tab1.index = pd.to_datetime(r16_tab1.index)

def create_features(r16_tab1):
            """
            Create time series features based on time series index.
            """
            r16_tab1 = r16_tab1.copy()
            r16_tab1['heure_mesure'] = r16_tab1.index.hour
            r16_tab1['minute_mesure'] = (r16_tab1.index.minute)
        #     r16_tab1['seconde_mesure'] = r16_tab1.index.second
            r16_tab1['depart'] =r16_tab1.depart
            r16_tab1['type_mesure'] = r16_tab1.type_mesure
            r16_tab1['phase'] = r16_tab1.phase
            return r16_tab1

test = create_features(r16_tab1)

FEATURES = ['depart', 'type_mesure', 'phase', 'heure_mesure', 'minute_mesure']
TARGET = 'mesure'

# X_train = train[FEATURES]
# y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

with open('r16_tab1_tdh_tension.pkl', "rb") as f:
    reg = pickle.load(f)

test['prediction'] = reg.predict(X_test)
test

In [ ]:
mere = test.copy()
mere['laisse'] = 'seconde_mesure'
test

In [ ]:
mon_dictionnaire = {'courant': int(1), 'courant_hta': int(2), 'desequilibre_courant':int(3),  
                    'tension_n':int(4), 'tension_ph':int(5), 'tension_ph_hta':int(6), 
                    'desequilibre_tension':int(7)}


In [ ]:
r16_tab1.loc[r16_tab1['type_mesure']== 'tdh_courant'].drop_duplicates(subset='depart')

In [ ]:
r16_tab1.loc[(r16_tab1['type_mesure'] == 'tdh_courant')]

In [ ]:
bt= r16_tab1.copy
bt
bt['type_mesure'].loc[bt['type_mesure']== 1] = eng.predict(bt)


In [4]:
r16_tab1.loc[r16_tab1['type_mesure']=='max_courant']

,type_mesure,phase,mesure,date_heure_mesure
57216,max_courant,1,0.90,2022-11-15 03:29:25
57218,max_courant,3,0.93,2022-11-15 03:29:25
57260,max_courant,2,1.07,2022-11-15 03:29:25
57277,max_courant,N,0.90,2022-11-15 03:29:25
154386,max_courant,2,1.07,2022-11-15 09:29:25
154423,max_courant,N,0.90,2022-11-15 09:29:25
154429,max_courant,1,0.90,2022-11-15 09:29:25
154453,max_courant,3,0.93,2022-11-15 09:29:25
154458,max_courant,N,0.90,2022-11-15 09:29:29
251153,max_courant,2,1.07,2022-11-15 15:29:25


In [29]:
r16_tab1.loc[r16_tab1['type_mesure']=='min_puissance_reactive']


,type_mesure,phase,mesure,date_heure_mesure
57224,min_puissance_reactive,3,4597.175,2022-11-15 03:29:25
57229,min_puissance_reactive,NaN,14067.074,2022-11-15 03:29:25
57237,min_puissance_reactive,2,4374.618,2022-11-15 03:29:25
57268,min_puissance_reactive,1,2799.049,2022-11-15 03:29:25
154397,min_puissance_reactive,2,2590.621,2022-11-15 09:29:25
154437,min_puissance_reactive,1,1138.696,2022-11-15 09:29:25
154438,min_puissance_reactive,3,1607.363,2022-11-15 09:29:25
154446,min_puissance_reactive,NaN,7484.541,2022-11-15 09:29:25
251141,min_puissance_reactive,2,2508.193,2022-11-15 15:29:25
251167,min_puissance_reactive,1,1104.547,2022-11-15 15:29:25


In [2]:
import itertools
import numpy as np
import pandas as pd
import random

a = [[1,2,3,4,5,6,7,8,9,10,11],[1,2,3,4,5]]
famille = list(itertools.product(*a))

my_data = []
for i in famille:
    my_data.append(list(i))

my_data = np.array(my_data)

column_names = ["type_mesure", "phase"]  #4,10 -- 5,11,6
data_df = pd.DataFrame(my_data, columns=column_names)
data = data_df.drop((data_df.loc[(data_df['type_mesure'].isin([6,11])) & (data_df['phase'].isin([1,2,3,4]))].index) |
                    (data_df.loc[(data_df['type_mesure'].isin([4,10])) & (data_df['phase'].isin([4]))].index) |
                    (data_df.loc[(data_df['type_mesure'].isin([1,7])) & (data_df['phase'].isin([5]))].index) |
                    (data_df.loc[(data_df['type_mesure'].isin([2,3,5,8,9])) & (data_df['phase'].isin([4,5]))].index)
)
data

C:\Users\user\AppData\Local\Temp\ipykernel_6204\2176784263.py:17: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  data = data_df.drop((data_df.loc[(data_df['type_mesure'].isin([6,11])) & (data_df['phase'].isin([1,2,3,4]))].index) |


,type_mesure,phase
0,1,1
1,1,2
2,1,3
3,1,4
5,2,1
6,2,2
7,2,3
10,3,1
11,3,2
12,3,3


In [3]:
import datetime
hr = int(datetime.datetime.now().strftime("%H"))
mn = int(datetime.datetime.now().strftime("%M"))
sd = int(datetime.datetime.now().strftime("%S"))
da_hr_ms = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
 
data['heure_mesure'] = hr
# data['minute_mesure'] = mn
# data['seconde_mesure'] = sd

In [ ]:
mon_dictionnaire = {'courant': int(1), 'courant_hta': int(2), 'desequilibre_courant':int(3),  
                    'tension_n':int(4), 'tension_ph':int(5), 'tension_ph_hta':int(6), 
                    'desequilibre_tension':int(7)}


In [4]:
import pickle

with open('r16_tfo1_max_courant.pkl', "rb") as a:
    Mcrt = pickle.load(a)
with open('r16_tfo1_max_tension_n.pkl', "rb") as b:
    Mten_n = pickle.load(b)
with open('r16_tfo1_max_tension_ph.pkl', "rb") as c:
    Mten_ph = pickle.load(c)
with open('r16_tfo1_max_puissance_reactive.pkl', "rb") as d:
    Mpui_rec = pickle.load(d)
with open('r16_tfo1_max_puissance_active.pkl', "rb") as e:
    Mpui_act = pickle.load(e)
with open('r16_tfo1_max_puissance_apparente.pkl', "rb") as f:
    Mpui_app = pickle.load(f)
with open('r16_tfo1_min_courant.pkl', "rb") as g:
    mcrt = pickle.load(g)
with open('r16_tfo1_min_tension_n.pkl', "rb") as h:
    mten_n = pickle.load(h)
with open('r16_tfo1_min_tension_ph.pkl', "rb") as i:
    mten_ph = pickle.load(i)
with open('r16_tfo1_min_puissance_reactive.pkl', "rb") as j:
    mpui_rec = pickle.load(j)
with open('r16_tfo1_min_puissance_apparente.pkl', "rb") as k:
    mpui_app = pickle.load(k)

data_omo = data.copy()

data['type_mesure'].loc[data['type_mesure']==1] = Mcrt.predict(data_omo.loc[data['type_mesure']==1])
data['type_mesure'].loc[data['type_mesure']==2] = Mten_n.predict(data_omo.loc[data_omo['type_mesure']== 2])
data['type_mesure'].loc[data['type_mesure']==3] = Mten_ph.predict(data_omo.loc[data_omo['type_mesure']== 3])
data['type_mesure'].loc[data['type_mesure']==4] = Mpui_rec.predict(data_omo.loc[data_omo['type_mesure']== 4])
data['type_mesure'].loc[data['type_mesure']==5] = Mpui_act.predict(data_omo.loc[data_omo['type_mesure']== 5])
data['type_mesure'].loc[data['type_mesure']==6] = Mpui_app.predict(data_omo.loc[data_omo['type_mesure']== 6])
data['type_mesure'].loc[data['type_mesure']==7] = mcrt.predict(data_omo.loc[data_omo['type_mesure']== 7])
data['type_mesure'].loc[data['type_mesure']==8] = mten_n.predict(data_omo.loc[data_omo['type_mesure']== 8])
data['type_mesure'].loc[data['type_mesure']==9] = mten_ph.predict(data_omo.loc[data_omo['type_mesure']== 9])
data['type_mesure'].loc[data['type_mesure']==10] = mpui_rec.predict(data_omo.loc[data_omo['type_mesure']== 10])
data['type_mesure'].loc[data['type_mesure']==11] = mpui_app.predict(data_omo.loc[data_omo['type_mesure']== 11])


C:\Users\user\AppData\Local\Temp\ipykernel_6204\1743507256.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['type_mesure'].loc[data['type_mesure']==1] = Mcrt.predict(data_omo.loc[data['type_mesure']==1])


In [ ]:
mon_dictionnaire = {'max_courant': int(1), 'max_tension_n': int(2), 'max_tension_ph':int(3),  
                    'max_puissance_reactive':int(4), 'max_puissance_active':int(5), 'max_puissance_apparente':int(6), 
                    'min_courant':int(7), 'min_tension_n':int(8), 'min_tension_ph':int(9), 'min_puissance_reactive':int(10),
                    'min_puissance_apparente':int(11)
                    }
        

In [5]:
data['mesure'] = data['type_mesure']
data['type_mesure'] = data_omo['type_mesure']

In [7]:
data['type_mesure'] = data['type_mesure'].map({
                            1:'max_courant', 2:'max_tension_n', 3:'max_tension_ph', 
                            4:'max_puissance_reactive', 5:'max_puissance_active', 6:'max_puissance_apparente',
                            7:'min_courant', 8:'min_tension_n', 9:'min_tension_ph', 10:'min_puissance_reactive',
                            11:'min_puissance_apparente'                   
                             },
                             na_action=None)

data['phase']= data['phase'].replace([4, 5],['N', ''])


In [9]:
data['date_heure_mesure'] = da_hr_ms
data = data.drop(columns=['heure_mesure'])


In [10]:

data['id_poste'] = 'P0016.210.043'
data['zone_poste'] = 'transfo1'
data['classification'] = 'CLA_BT'
# data['classification'].loc[data['type_mesure'].isin(['tension_ph_hta', 'courant_hta'])] = 'CLA_HTA'
data['date_heure_transfert'] = da_hr_ms
data['unite_mesure'] = data['type_mesure']
data['unite_mesure'].loc[data['unite_mesure'].isin(['max_courant', 'min_courant', '', ''])] = 'V'
data['unite_mesure'].loc[data['unite_mesure'].isin(['max_tension_n', 'min_tension_n', 'max_tension_ph', 'min_tension_ph'])] = 'A'
data['unite_mesure'].loc[data['unite_mesure'].isin(['min_puissance_active', 'max_puissance_active', '', '',''])] = 'W'
data['unite_mesure'].loc[data['unite_mesure'].isin(['max_puissance_apparente'])] = 'VA'
data['unite_mesure'].loc[data['unite_mesure'].isin(['max_puissance_reactive', 'min_puissance_reactive'])] = 'VAR'
# data['unite_mesure'].loc[data['unite_mesure'].isin(['facteur_puissance', 'taux_de_charge', 'tdh_courant', 'tdh_tension'])] = '%'
# data['phase'] = data['phase'].astype(int)
data['phase'] = data['phase'].astype(str)
data['description_mesure'] = 'mesure - ' + data['type_mesure'] + ' phase ' +  data['phase']


C:\Users\user\AppData\Local\Temp\ipykernel_6204\2167375344.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['unite_mesure'].loc[data['unite_mesure'].isin(['max_courant', 'min_courant', '', ''])] = 'V'
C:\Users\user\AppData\Local\Temp\ipykernel_6204\2167375344.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['unite_mesure'].loc[data['unite_mesure'].isin(['max_tension_n', 'min_tension_n', 'max_tension_ph', 'min_tension_ph'])] = 'A'
C:\Users\user\AppData\Local\Temp\ipykernel_6204\2167375344.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in 

In [ ]:
data

In [ ]:
data = data[[
    'id_poste', 'zone_poste', 'classification', 
    'type_mesure', 'description_mesure', 'phase', 'mesure',
    'unite_mesure', 'date_heure_mesure','date_heure_transfert'
    ]]
    
data_log = data.to_json(orient = 'records', lines=True)


In [ ]:
files_name = 'R16_Tab1_{}.log'.format(datetime.datetime.now().strftime("%Y-%m-%d"))

file = open(files_name, "a+")
file.write(data_log)
file.close()


In [ ]:
test['prediction'] = reg.predict(X_test)


In [ ]:
test.loc[(test['type_mesure']==1) & (test['depart']==2)].iloc[0:60]
